<a href="https://colab.research.google.com/github/DGuilherme/PMTese/blob/main/CNNTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install tensorflow scikit-learn

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import keras
from keras import layers

from google.colab import drive
drive.mount('/content/drive')

# Load data
column_names = ['id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f'sensor{i}' for i in range(1, 22)]
train_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/train_FD001.txt/train_FD001.txt', delim_whitespace=True, header=None)
test_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/test_FD001.txt/test_FD001.txt', delim_whitespace=True, header=None)
rul_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/RUL_FD001.txt/RUL_FD001.txt', delim_whitespace=True, header=None)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-16-60e5bacaffeb>:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  train_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/train_FD001.txt/train_FD001.txt', delim_whitespace=True, header=None)
<ipython-input-16-60e5bacaffeb>:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  test_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/test_FD001.txt/test_FD001.txt', delim_whitespace=True, header=None)
<ipython-input-16-60e5bacaffeb>:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  rul_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/RUL_FD001.txt/RUL_FD001.t

In [18]:
train_data.columns = column_names
test_data.columns = column_names
rul_data.columns = ['RUL']


# Generate RUL for training data
max_cycle = train_data.groupby('id')['cycle'].max()
train_data = train_data.merge(max_cycle.reset_index(), on='id', suffixes=('', '_max'))
train_data['RUL'] = train_data['cycle_max'] - train_data['cycle']
train_data.drop(columns=['cycle_max'], inplace=True)

# RUL for test data
max_cycle_test = test_data.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id', 'max_cycle']
rul_data.columns = ['RUL']
max_cycle_test['RUL'] = rul_data['RUL']
test_data = test_data.merge(max_cycle_test, on='id')
test_data['RUL'] = test_data['RUL'] + test_data['max_cycle'] - test_data['cycle']
test_data.drop(columns=['max_cycle'], inplace=True)

useful_sensor_cols = ['setting1', 'setting2', 'setting3'] + \
    [f'sensor{i}' for i in [2, 3, 4, 7, 8, 11, 12, 13, 14, 15, 17, 20, 21]]

train_data = train_data[['id', 'cycle'] + useful_sensor_cols + ['RUL']]
test_data = test_data[['id', 'cycle'] + useful_sensor_cols + ['RUL']]

# Normalize sensor data
scaler = MinMaxScaler()
train_data.iloc[:, :17] = scaler.fit_transform(train_data.iloc[:, :17])
test_data.iloc[:, :17] = scaler.transform(test_data.iloc[:, :17])

<ipython-input-18-38aadda1c23d>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 1. 1. 1.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.iloc[:, :17] = scaler.fit_transform(train_data.iloc[:, :17])
<ipython-input-18-38aadda1c23d>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.         0.00277008 0.00554017 ... 0.54570637 0.54847645 0.55124654]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.iloc[:, :17] = scaler.fit_transform(train_data.iloc[:, :17])
<ipython-input-18-38aadda1c23d>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.33333333 0.33333333 0.16666667 ... 0.83333333 0.58333333 0.66666667]' has dtype incompatible with int64, please exp

In [27]:
# Reshape data for CNN input (assuming 2D input)
train_data = train_data.values.reshape(train_data.shape[0], train_data.shape[1], 1)
test_data = test_data.values.reshape(test_data.shape[0], test_data.shape[1], 1)

In [12]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model

In [14]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
        # Normalization and Attention
        x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
        x = LayerNormalization(epsilon=1e-6)(inputs)
        x = Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
        x = layers.Dropout(dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        return x + res

In [15]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [28]:
print(train_data.shape[1:])

print(train_data[:, :1])

(19, 1)
[[[0.]]

 [[0.]]

 [[0.]]

 ...

 [[1.]]

 [[1.]]

 [[1.]]]


In [ ]:
input_shape = train_data.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    train_data[:, :],
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)